<a href="https://colab.research.google.com/github/ezgior/Persona/blob/main/Hybrid_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hybrid Recommendation System

In [1]:
import pandas as pd

In [4]:
def create_user_movie_df():
    import pandas as pd
    movie = pd.read_csv('movie.csv')
    rating = pd.read_csv('rating.csv')
    df = movie.merge(rating, how="left", on="movieId")
    comment_counts = pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 5000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df

user_movie_df = create_user_movie_df()

In [5]:
user_movie_df.head()

title,(500) Days of Summer (2009),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),"13th Warrior, The (1999)","20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),21 Grams (2003),28 Days Later (2002),300 (2007),"40-Year-Old Virgin, The (2005)",50 First Dates (2004),8 Mile (2002),A.I. Artificial Intelligence (2001),About Schmidt (2002),About a Boy (2002),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Adaptation (2002),Addams Family Values (1993),"Addams Family, The (1991)","Adventures of Priscilla, Queen of the Desert, The (1994)","African Queen, The (1951)",Air Force One (1997),Airheads (1994),Airplane! (1980),Akira (1988),Aladdin (1992),Alice in Wonderland (1951),Alien (1979),Alien: Resurrection (1997),Aliens (1986),Alien³ (a.k.a. Alien 3) (1992),Almost Famous (2000),Amadeus (1984),"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",American Beauty (1999),American Gangster (2007),...,Waterworld (1995),Wayne's World (1992),Wayne's World 2 (1993),Wedding Crashers (2005),"Wedding Singer, The (1998)",Weekend at Bernie's (1989),Weird Science (1985),Welcome to the Dollhouse (1995),West Side Story (1961),What About Bob? (1991),What Lies Beneath (2000),What Women Want (2000),What's Eating Gilbert Grape (1993),When Harry Met Sally... (1989),While You Were Sleeping (1995),White Men Can't Jump (1992),Who Framed Roger Rabbit? (1988),"Whole Nine Yards, The (2000)",Wild Things (1998),Wild Wild West (1999),William Shakespeare's Romeo + Juliet (1996),Willow (1988),Willy Wonka & the Chocolate Factory (1971),Witness (1985),"Wizard of Oz, The (1939)",Wolf (1994),Wonder Boys (2000),"World Is Not Enough, The (1999)",Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men: The Last Stand (2006),X2: X-Men United (2003),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Zombieland (2009),Zoolander (2001),eXistenZ (1999),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,NaN,4.0,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
user_movie_df["10 Things I Hate About You (1999)"]

userId
1.0        NaN
2.0        NaN
3.0        NaN
4.0        NaN
5.0        NaN
            ..
138489.0   NaN
138490.0   NaN
138491.0   NaN
138492.0   NaN
138493.0   NaN
Name: 10 Things I Hate About You (1999), Length: 138476, dtype: float64

Movies watched by the user we select 

In [9]:
user =108170

In [10]:
user_df = user_movie_df[user_movie_df.index == user]
movies_watched = user_df.columns[user_df.notna().any()].tolist()

In [11]:
len(movies_watched)

166

In [12]:


movies_watched_df = user_movie_df[movies_watched]
user_movie_count = movies_watched_df.T.notnull().sum()



In [13]:
user_movie_count = user_movie_count.reset_index()

user_movie_count.head()

,userId,0
0,1.0,52
1,2.0,10
2,3.0,47
3,4.0,5
4,5.0,16


In [14]:
user_movie_count.columns = ["userId", "movie_count"]
user_movie_count[user_movie_count["movie_count"] > 20].sort_values("movie_count", ascending=False)


,userId,movie_count
108159,108170.0,166
118191,118205.0,166
8403,8405.0,165
29570,29575.0,165
118740,118754.0,165
...,...,...
46720,46728.0,21
31292,31297.0,21
31286,31291.0,21
112074,112086.0,21


In [15]:
user_movie_count[user_movie_count["movie_count"] == len(movies_watched)].count()
users_same_movies = user_movie_count[user_movie_count["movie_count"] > len(movies_watched)*0.8]["userId"]
users_same_movies.head()

90      91.0
585    586.0
740    741.0
774    775.0
970    971.0
Name: userId, dtype: float64

In [16]:
users_same_movies.count() 

527

Now we we will determine users have watched same movies with the user_id =1 

1.   In order for it, we will combine data of our user with the others 
2.   We will create correlation dataframe
Then find the most similar users 





In [17]:

final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                      user_df[movies_watched]])

In [18]:
final_df.head()

title,2001: A Space Odyssey (1968),"Adventures of Priscilla, Queen of the Desert, The (1994)",Akira (1988),Aladdin (1992),Aliens (1986),Almost Famous (2000),Amadeus (1984),American History X (1998),Arachnophobia (1990),Army of Darkness (1993),Basic Instinct (1992),Batman Returns (1992),"Big Lebowski, The (1998)","Blair Witch Project, The (1999)","Blues Brothers, The (1980)","Boondock Saints, The (2000)","Bourne Identity, The (2002)",Braveheart (1995),"Breakfast Club, The (1985)",Carlito's Way (1993),Charlie's Angels (2000),City Slickers (1991),City Slickers II: The Legend of Curly's Gold (1994),Clear and Present Danger (1994),Clerks (1994),"Clockwork Orange, A (1971)",Con Air (1997),Contact (1997),Copycat (1995),Crocodile Dundee (1986),Cruel Intentions (1999),"Crying Game, The (1992)","Dark Crystal, The (1982)",Dead Poets Society (1989),Deliverance (1972),Demolition Man (1993),Desperado (1995),"Devil's Advocate, The (1997)",Donnie Darko (2001),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),...,"Sound of Music, The (1965)","Specialist, The (1994)",Species (1995),Speed 2: Cruise Control (1997),Spider-Man 2 (2004),Stand by Me (1986),Star Trek II: The Wrath of Khan (1982),Star Trek III: The Search for Spock (1984),Star Trek: First Contact (1996),Star Trek: Generations (1994),Star Wars: Episode I - The Phantom Menace (1999),Star Wars: Episode II - Attack of the Clones (2002),Star Wars: Episode III - Revenge of the Sith (2005),Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back (1980),Star Wars: Episode VI - Return of the Jedi (1983),Terminator 2: Judgment Day (1991),"Terminator, The (1984)",There's Something About Mary (1998),"Three Musketeers, The (1993)",Tin Cup (1996),Titanic (1997),Tomorrow Never Dies (1997),Top Gun (1986),Toy Story (1995),Toy Story 2 (1999),Traffic (2000),Trainspotting (1996),"Truman Show, The (1998)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Twister (1996),"Untouchables, The (1987)","Usual Suspects, The (1995)",Wallace & Gromit: A Close Shave (1995),Wallace & Gromit: The Best of Aardman Animation (1996),Wallace & Gromit: The Wrong Trousers (1993),"Wedding Singer, The (1998)",Who Framed Roger Rabbit? (1988),Willow (1988),X2: X-Men United (2003)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
91.0,2.5,4.0,2.5,NaN,4.0,3.0,3.5,3.5,2.5,NaN,NaN,3.5,2.0,3.0,4.5,NaN,3.5,5.0,4.0,NaN,2.0,4.5,3.0,3.0,1.5,5.0,2.5,4.0,4.0,3.5,NaN,3.0,4.0,4.5,NaN,3.0,3.0,2.5,NaN,4.0,...,4.0,NaN,1.5,NaN,2.0,NaN,NaN,NaN,3.0,3.5,3.0,3.0,3.5,4.5,5.0,4.5,2.5,3.0,2.5,NaN,2.5,3.5,2.5,2.5,4.0,4.5,3.0,4.0,3.5,4.0,3.0,4.0,3.5,4.0,4.0,4.0,3.5,4.0,3.5,3.5
586.0,5.0,2.5,4.0,3.0,3.5,3.0,4.0,4.0,NaN,NaN,2.0,2.5,4.0,2.0,4.0,4.0,4.5,4.5,3.0,NaN,3.0,2.5,2.5,NaN,4.0,5.0,3.5,3.0,NaN,3.5,3.5,NaN,NaN,4.0,4.0,3.0,4.0,3.5,5.0,4.5,...,NaN,3.5,2.5,1.5,3.0,NaN,NaN,NaN,3.5,3.5,3.0,4.0,4.5,3.5,3.5,3.5,5.0,5.0,4.0,NaN,3.0,2.5,2.5,3.0,2.5,2.0,2.5,3.5,3.5,4.0,3.0,3.0,4.5,3.0,4.0,3.0,3.5,3.0,NaN,3.5
741.0,5.0,NaN,NaN,4.0,4.5,3.5,5.0,5.0,0.5,0.5,4.5,4.5,5.0,5.0,2.0,NaN,3.0,4.5,4.5,5.0,1.0,4.5,4.0,2.5,1.0,5.0,3.5,1.5,2.5,NaN,NaN,5.0,4.5,5.0,5.0,0.5,4.5,4.5,NaN,5.0,...,5.0,0.5,2.0,0.5,3.0,4.5,4.0,4.0,4.0,1.0,0.5,NaN,3.5,5.0,4.5,4.0,3.5,4.5,4.5,4.5,NaN,0.5,4.0,4.0,5.0,4.5,2.5,5.0,5.0,5.0,0.5,5.0,5.0,NaN,NaN,NaN,1.0,5.0,5.0,NaN
775.0,5.0,4.5,4.5,3.0,5.0,3.0,5.0,2.0,3.5,4.5,2.0,4.5,4.0,4.0,4.5,0.5,NaN,5.0,4.0,4.0,4.0,4.0,1.5,3.5,4.5,5.0,3.5,4.0,3.0,3.5,4.5,5.0,3.5,3.5,5.0,3.0,4.0,3.0,4.0,5.0,...,5.0,1.0,4.0,1.0,5.0,3.5,5.0,3.0,5.0,3.5,2.0,3.0,NaN,5.0,5.0,4.5,4.5,4.0,4.5,NaN,NaN,4.0,3.5,3.5,4.5,4.0,3.0,5.0,5.0,4.0,2.5,4.0,4.0,5.0,4.5,4.5,4.0,5.0,4.0,4.0
971.0,3.0,3.0,3.0,NaN,5.0,3.0,NaN,5.0,3.0,4.5,3.0,NaN,5.0,3.5,3.0,NaN,3.5,3.0,3.5,3.5,NaN,3.5,3.0,3.5,3.0,4.0,3.5,NaN,3.0,2.5,3.0,4.0,2.0,4.0,4.5,2.0,NaN,3.5,4.5,5.0,...,2.0,NaN,3.0,NaN,NaN,4.0,3.0,2.5,3.5,3.0,3.5,4.5,5.0,5.0,5.0,5.0,3.5,5.0,4.0,1.0,NaN,2.5,NaN,2.0,NaN,NaN,3.5,4.5,4.0,5.0,NaN,3.5,5.0,NaN,NaN,NaN,3.5,3.0,3.5,3

In [19]:
final_df.T.corr()

userId,91.0,586.0,741.0,775.0,971.0,982.0,1507.0,1748.0,1849.0,2099.0,2158.0,2261.0,2397.0,2669.0,3318.0,3487.0,3629.0,3907.0,4129.0,4222.0,4358.0,4507.0,4529.0,4594.0,4598.0,4696.0,5576.0,5768.0,5843.0,6099.0,6373.0,6976.0,6978.0,7370.0,7554.0,7828.0,7858.0,8405.0,8932.0,8966.0,...,127434.0,127450.0,128258.0,128690.0,129420.0,129540.0,130459.0,130516.0,130876.0,130987.0,131100.0,131180.0,131275.0,131347.0,131698.0,131961.0,132039.0,132185.0,133133.0,133703.0,133811.0,134175.0,134352.0,134774.0,135090.0,135425.0,135522.0,135795.0,136268.0,136599.0,136875.0,136957.0,136989.0,137037.0,137202.0,137277.0,137343.0,137686.0,138208.0,108170.0
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
91.0,1.000000,0.187587,0.405896,0.250998,0.271148,0.335445,0.348735,0.312729,0.463046,0.275244,0.298351,0.280982,0.058066,0.412660,0.276556,0.416262,0.133133,0.418799,0.289023,0.325950,0.224621,0.288956,0.170481,0.391549,0.425857,0.318493,0.542388,0.308119,0.253388,0.351846,0.482401,0.060400,0.347360,0.314026,0.518697,0.351421,0.238566,0.386191,0.182209,0.308865,...,0.347388,0.112205,0.337365,0.395905,0.286675,0.316949,0.233328,0.322579,0.408537,0.193375,0.415286,0.206944,0.369302,0.381460,0.324641,0.322914,0.135621,0.202757,0.372382,0.447330,0.250122,0.297093,0.409218,0.199733,0.462838,0.125006,0.328571,0.461079,0.037444,0.259682,0.116679,0.378949,0.145230,0.251997,0.358827,0.399961,0.383098,0.438142,0.514921,0.327649
586.0,0.187587,1.000000,0.373718,0.364255,0.466290,0.531234,0.340249,0.216173,0.285540,0.488748,0.288429,0.171061,-0.096305,0.542664,0.365437,0.351326,0.050293,0.321685,0.417890,0.180625,0.386906,0.390841,0.212142,0.228434,0.379750,0.338596,0.372537,0.558132,0.208975,0.450833,0.404082,0.100459,0.371608,0.379359,0.401219,0.304121,0.309525,0.313440,0.332529,0.294891,...,0.244130,0.364375,0.223565,0.502727,-0.187485,0.446786,0.252601,0.332017,0.333158,0.381617,0.321111,0.315961,0.223520,0.499649,0.440396,0.368384,0.500050,0.405593,0.572612,0.264982,0.165293,0.303090,0.468557,0.232579,0.435049,0.209572,0.469533,0.464395,0.132627,0.414176,0.488534,0.311704,0.175816,0.533230,0.398656,0.542488,0.485978,0.455062,0.325836,0.625304
741.0,0.405896,0.373718,1.000000,0.409519,0.354803,0.465765,0.357996,0.140730,0.421389,0.336902,0.326326,0.323081,0.142435,0.524866,0.253649,0.312270,0.203567,0.398838,0.386654,0.374797,0.455078,0.264141,0.321482,0.295081,0.396457,0.308769,0.363008,0.512973,0.333604,0.556000,0.382843,0.108205,0.473621,0.310728,0.313277,0.197032,0.265393,0.459389,0.348234,0.441608,...,0.265046,0.392370,0.216164,0.408569,0.102901,0.544859,0.204823,0.352977,0.287613,0.278509,0.528909,0.212129,0.327323,0.457563,0.447121,0.492850,0.289813,0.451242,0.505154,0.389094,0.196120,0.386847,0.398910,0.115443,0.508527,0.296419,0.393171,0.351282,0.155077,0.339261,0.454752,0.324540,0.310529,0.536549,0.507600,0.542344,0.134766,0.458671,0.376968,0.395863
775.0,0.250998,0.364255,0.409519,1.000000,0.267275,0.426600,0.261580,0.118477,0.289109,0.318967,0.307256,0.287569,0.021497,0.251371,0.272050,0.213561,0.377394,0.506460,0.353811,0.400490,0.403582,0.493718,0.343734,0.427481,0.291395,0.536015,0.448622,0.515210,0.390306,0.430799,0.274954,0.235829,0.451157,0.346762,0.302532,0.240604,0.240061,0.586270,0.373702,0.275548,...,0.400256,0.409252,0.091692,0.467878,0.129779,0.276231,0.221845,0.326549,0.314849,0.358879,0.390284,0.398805,0.231837,0.448294,0.390048,0.528388,0.183854,0.326080,0.290452,0.334356,0.220050,0.318743,0.262984,0.144496,0.374199,0.404293,0.318125,0.339081,0.348501,0.296331,0.309641,0.268362,0.348335,0.424969,0.350220,0.438156,0.291697,0.619776,0.390387,0.337146
971.0,0.271148,0.466290,0.354803,0.267275,1.000000,0.548980,0.296572,0.223739,0.329604,0.439593,0.276433,0.026533,0.108511,0.449465,0.292103,0.188407,0.117403,0.438931,0.384392,0.321499,0.305550,0.422337,0.334356,0.420033,0.426003,0.407610,0.523321,0.548356,0.283207,0.526652,0.323552,0.168049,0.249972,0.457407,0.425044,0.157002,0.246581,0.397783

In [20]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()

In [21]:
corr_df = pd.DataFrame(corr_df, columns=["corr"])


In [22]:
corr_df.index

MultiIndex([( 60159.0,  43296.0),
            ( 76540.0,  21078.0),
            (116317.0,  44530.0),
            (116317.0,  43296.0),
            ( 76540.0, 122364.0),
            ( 40285.0, 107640.0),
            ( 98687.0,  76540.0),
            ( 98687.0,  44530.0),
            (116317.0,  76540.0),
            ( 40285.0, 129420.0),
            ...
            ( 17918.0,   8405.0),
            (100992.0,   8405.0),
            ( 29056.0,  29037.0),
            (  8405.0,  25478.0),
            ( 56504.0,  56720.0),
            ( 25478.0, 100992.0),
            ( 17918.0, 100992.0),
            ( 17918.0,  25478.0),
            ( 32344.0,  68063.0),
            (108170.0, 108170.0)],
           names=['userId', 'userId'], length=138602)

In [23]:
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df.head()

corr
user_id_1 user_id_2          
60159.0   43296.0   -0.323455
76540.0   21078.0   -0.296238
116317.0  44530.0   -0.281763
          43296.0   -0.266501
76540.0   122364.0  -0.264063

In [24]:
corr_df = corr_df.reset_index()

In [25]:
top_users = corr_df[(corr_df["user_id_1"] == user) & (corr_df["corr"] >= 0.55)][
    ["user_id_2", "corr"]].reset_index(drop=True)

In [26]:
top_users = top_users.sort_values(by='corr', ascending=False)
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)


In [27]:
top_users.head()

,userId,corr
15,108170.0,1.000000
14,118254.0,0.644375
13,58170.0,0.608997
12,2669.0,0.602224
11,51227.0,0.595812


In [28]:
rating = pd.read_csv('rating.csv')
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')

In [29]:
top_users_ratings = top_users_ratings[top_users_ratings["userId"] != user]

In [30]:
top_users_ratings.head()

,userId,corr,movieId,rating
194,118254.0,0.644375,1,2.0
195,118254.0,0.644375,2,2.5
196,118254.0,0.644375,3,2.5
197,118254.0,0.644375,6,3.5
198,118254.0,0.644375,10,3.0


In [31]:
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
top_users_ratings.head()

,userId,corr,movieId,rating,weighted_rating
194,118254.0,0.644375,1,2.0,1.288751
195,118254.0,0.644375,2,2.5,1.610939
196,118254.0,0.644375,3,2.5,1.610939
197,118254.0,0.644375,6,3.5,2.255314
198,118254.0,0.644375,10,3.0,1.933126


In [32]:
top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})


,weighted_rating
movieId,
1,2.029113
2,1.720057
3,1.484377
5,1.072693
6,2.123694
...,...
118696,1.928330
118700,1.928330
118702,1.928330


In [33]:
recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})

recommendation_df = recommendation_df.reset_index()

In [34]:
recommendation_df.head()

,movieId,weighted_rating
0,1,2.029113
1,2,1.720057
2,3,1.484377
3,5,1.072693
4,6,2.123694


In [35]:
recommendation_df[["movieId"]].nunique()

movieId    5386
dtype: int64

In [36]:
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3].sort_values("weighted_rating", ascending=False)


In [37]:
movies_to_be_recommend.head()

,movieId,weighted_rating


In [38]:
movie = pd.read_csv('movie.csv')

In [39]:
weighted_recommendation = movies_to_be_recommend.merge(movie[["movieId", "title"]])["title"]

In [41]:
weighted_recommendation.head()

Series([], Name: title, dtype: object)

In [42]:
movie_id =rating[(rating["userId"]==user)& (rating["rating"]==5.0)].\
              sort_values(by="timestamp",ascending= False)["movieId"][0:1]
movie_id = movie_id.iloc[0]


In [43]:
def check__movie_id(movie_id):
    movie = pd.read_csv('movie.csv')
    movie_name = movie[movie["movieId"] == movie_id][["title"]].values[0].tolist()
    return movie_name

In [44]:
movie_name =check__movie_id(movie_id)

In [45]:
movie_name

['Wild at Heart (1990)']